In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,precision_score,recall_score, f1_score,roc_auc_score,roc_curve,balanced_accuracy_score,confusion_matrix


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

df = pd.read_csv('/content/drive/MyDrive/Database System /Insurance Project/diabetes_012_health_indicators_BRFSS2015.csv')

df.rename(columns={'Diabetes_012':'Diabetes_binary'}, inplace=True)



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_binary       253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [ ]:
# HASN'T FALLEN SICK IN THE LAST 30 DAYS AND FEELS EXCELLENT OR VERY GOOD
df['PerfectHlth'] = (df['PhysHlth']==0.0) & (df['GenHlth']<3.0)
df['PerfectHlth'] = df['PerfectHlth'].astype(int)

In [ ]:
# No single issue with heart disease or BP
df['NoHrtIssue'] = (df['HighBP']==0.0) & (df['HighChol']==0.0) & (df['HeartDiseaseorAttack']==0.0)
df['NoHrtIssue'] = df['NoHrtIssue'].astype(int)


In [ ]:
# Diabetics would most likely feel unwell and in a slightly poorer state of health
df['hlthNotGood'] = df['GenHlth'] > 3
df['HlthAboveGood'] = df['GenHlth'] < 3
df['hlthNotGood'] = df['hlthNotGood'].astype(int)
df['HlthAboveGood'] = df['HlthAboveGood'].astype(int)


In [ ]:
# people with high blood pressure and heart disease/illness
df['hbp&HA'] = (df['HeartDiseaseorAttack'] == 1.0) & (df['HighBP'] == 1.0)
df['hbp&HA'] = df['hbp&HA'].astype(int)
df['older&poor'] = (df['Age'] > 5.0) & (df['Income'] < 5)
df['older&poor'] = df['older&poor'].astype(int)

In [ ]:
# drop irrelevant columns based on correlation
df2 = df.drop(['Fruits', 'Veggies', 'AnyHealthcare', 'NoDocbcCost', 'Income',
              'Education', 'CholCheck'],axis=1)
df3 = df2.copy()

In [ ]:
sc =StandardScaler()

df3[['BMI']] = sc.fit_transform(df3[['BMI']])
df3[['Age']] = sc.fit_transform(df3[['Age']])
df3[['PhysHlth']] = sc.fit_transform(df3[['PhysHlth']])
df3[['MentHlth']] = sc.fit_transform(df3[['MentHlth']])


QT=QuantileTransformer(n_quantiles=500,output_distribution='normal')

df3[['BMI']] = QT.fit_transform(df3[['BMI']])
df3[['Age']] = QT.fit_transform(df3[['Age']])
df3[['PhysHlth']] = QT.fit_transform(df3[['PhysHlth']])
df3[['MentHlth']] = QT.fit_transform(df3[['MentHlth']])


In [ ]:
X = df3.drop('Diabetes_binary', axis =1)
y = df3['Diabetes_binary']
OverS=RandomOverSampler(random_state=99, sampling_strategy='not majority')
X_over,y_over=OverS.fit_resample(X,y)
X_over_train,X_over_test,y_over_train,y_over_test=train_test_split(X_over,y_over,test_size=0.2,random_state=99)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 10, 15, 20]
}


RF_grid_search_ = GridSearchCV(rfc, param_grid=param_grid, cv=3, n_jobs=-1,verbose=3)
RF_grid_search_.fit(X_over_train,y_over_train)

HighBP  HighChol       BMI  Smoker  Stroke  HeartDiseaseorAttack  \
0            1         1  1.615454       1       0                     0   
1            0         0 -0.494090       1       0                     0   
2            1         1  0.141119       0       0                     0   
3            1         0 -0.065349       0       0                     0   
4            1         1 -0.717694       0       0                     0   
...        ...       ...       ...     ...     ...                   ...   
398800       1         1  0.141119       1       0                     0   
398801       1         1  0.141119       0       0                     0   
398802       1         0  0.141119       0       0                     1   
398803       1         0 -0.065349       1       0                     1   
398804       1         1 -0.290357       1       0                     0   

        PhysActivity  HvyAlcoholConsump  GenHlth  MentHlth  PhysHlth  \
0                  0                  0        4  1.499023  1.220053   
1                  1                  0        2 -5.199338 -5.199338   
2                  0                  0        4  5.199338  5.199338   
3                  1                  0        1 -5.199338 -5.199338   
4                  1                  0        1  0.822449 -5.199338   
...              ...                ...      ...       ...       ...   
398800             0                  0        3 -5.199338  5.199338   
398801             1                  0        2 -5.199338 -5.199338   
398802             0                  0        3 -5.199338 -5.199338   
398803             0                  0        4  1.410756  5.199338   
398804             0                  0        2 -5.199338  1.358362   

        DiffWalk  Sex       Age  PerfectHlth  NoHrtIssue  hlthNotGood  \
0              1    0  0.199741            0           0            1   
1              0    0 -0.424306            0           1            0   
2              1    0  0.199741            0           0            1   
3              0    0  0.921646            1           0            0   
4              0    0  0.921646            1           0            0   
...          ...  ...       ...          ...         ...          ...   
398800         1    0  0.557546            0           0            1   
398801         1    1  0.557546            0           0            0   
398802         0    1  1.280411            0           0            1   
398803         1    0 -0.133513            0           0            1   
398804         1    0  0.921646            0           0            0   

        HlthAboveGood  hbp&HA  older&poor  
0                   0       0           1  
1                   0       0           1  
2                   0       0           0  
3                   1       0           0  
4                   1       0           1  
...               ...     ...         ...  
398800              0       0           0  
398801              0       0           0  
398802              0       1           1  
398803              0       1           1  
398804              0       0           0  

[398805 rows x 20 columns]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5, 10, 15, 20]
}


RF_grid_search_ = GridSearchCV(rfc, param_grid=param_grid, cv=3, n_jobs=-1,verbose=3)
RF_grid_search_.fit(X_over_train,y_over_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 20],
                         'n_estimators': [50, 100, 200, 300]},
             verbose=3)

In [ ]:
# Get the best parameters from the grid search
best_params = RF_grid_search_.best_params_
print("Best parameters found: ", best_params)

# Use the best parameters to train a new RandomForestClassifier
best_rfc = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'])
best_rfc.fit(X_over_train, y_over_train)

Best parameters found:  {'max_depth': 20, 'n_estimators': 300}


RandomForestClassifier(max_depth=20, n_estimators=300)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score, roc_auc_score
import pandas as pd

# Predict on the test set
y_pred = best_rfc.predict(X_over_test)

# Calculate metrics
accuracy = accuracy_score(y_over_test, y_pred)
f1 = f1_score(y_over_test, y_pred, average='weighted')
precision = precision_score(y_over_test, y_pred, average='weighted')
recall = recall_score(y_over_test, y_pred, average='weighted')
balanced_accuracy = balanced_accuracy_score(y_over_test, y_pred)




# Create a DataFrame to display the results
metrics_data = {
    'Metric': ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'Balanced Accuracy'],
    'Value': [accuracy, f1, precision, recall, balanced_accuracy]
}

df_metrics = pd.DataFrame(metrics_data)

print(df_metrics)


              Metric     Value
0           Accuracy  0.891325
1           F1 Score  0.889438
2          Precision  0.898578
3             Recall  0.891325
4  Balanced Accuracy  0.891223
